*`torchvision` - base domain library for Pytorch computer vision
*`torchvision.datasets` - get datasets and data loading functions for computer vision here
*`torchvision.models` - get pretrained computer vision models that you can leverage or your own problems
*`torchvision.transforms` - functions for manipulating your vision data (images) into suitable for use in model
*`torch.utils.data.Dataset` - Base dataset class for pytorch
*`torch.utils.data.DataLoader` - creates a python interable over a dataset

In [1]:
import torch 
from torch import nn

import torchvision 
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [2]:
torch.__version__, torchvision.__version__

('2.9.1+cu128', '0.24.1+cu128')

In [ ]:
# Getting a dataset
# Fashion MNIST dataset

train_dataset = datasets.FashionMNIST(
    root="data",
    train=True, # train data
    download=True,
    transform=ToTensor(), # how to transform the data
    target_transform=None
)

test_dataset = datasets.FashionMNIST(
    root="data",
    train=False, # train data
    download=True,
    transform=ToTensor(), # how to transform the data
    target_transform=None
)

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
class_names = train_dataset.classes
class_names

In [ ]:
train_dataset.class_to_idx

In [ ]:
train_dataset.targets

In [ ]:
image = train_dataset[0][0]
image

In [ ]:
image.shape

In [ ]:
# visualise
image, label = train_dataset[0]
plt.imshow(image.squeeze())
plt.title(label)

In [ ]:
plt.imshow(image.squeeze(), cmap="gray")

In [ ]:
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols = 4, 4

for i in range(1, rows*cols+1):
    random_idx = torch.randint(0, len(train_dataset), size=[1]).item()
    image, label = train_dataset[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(image.squeeze(), cmap="gray")
    plt.title(label)
    plt.axis(False)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=32,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=32,
                             shuffle=False)

In [ ]:
print(f"Dataloaders: {train_dataset, test_dataloader}")
print(f"Length of train_dataloader: {len(train_dataloader)} batches of 32")
print(f"Length of test_dataloader: {len(test_dataloader)} batches of 32")

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(label)
plt.axis(False)

In [ ]:
flatten_model = nn.Flatten()

x = train_features_batch[0]

output = flatten_model(x)

x.shape, output.shape

In [ ]:
from torch import nn
class FashionModel(nn.Module):
    def __init__(self, 
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape,
                      out_features=hidden_units,),
            nn.Linear(in_features=hidden_units,
                      out_features=output_shape)
        )

    def forward(self,x):
        return self.layer_stack(x)

In [ ]:
import torch
torch.manual_seed(42)

model_0 = FashionModel(
    input_shape=28*28,
    hidden_units=10,
    output_shape=10
).to("cpu")

model_0

In [ ]:
dummy_x = torch.rand([1,1,28,28])
print(f"{model_0(dummy_x).squeeze()} -> logits")

In [ ]:
import requests 
from pathlib import Path

if Path("helper_functions.py").is_file():
    print("already exists")
else:
    print("Downloading helper_functions.py")
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py")
    with open("helper_functions.py", "wb") as f:
        f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                         lr=0.1)

In [ ]:
from timeit import default_timer as timer 

def print_train_time(start:float,
                     end: float,
                     device: torch.device= None):
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [ ]:
# training model
from tqdm.auto import tqdm

torch.manual_seed(42)
start = timer()

epochs=3

for epoch in tqdm(range(epochs)):
    print(f"EpochL {epoch} \n-----")

    train_loss = 0

    for batch, (X,y) in enumerate(train_dataloader):
        model_0.train()
        y_pred = model_0(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        if batch % 400 == 0:
            print(f"Trained on {batch * len(X)}/{len(train_dataloader.dataset)} samples")
    
    train_loss /= len(train_dataloader)

    test_loss, test_acc = 0, 0
    model_0.eval()

    with torch.inference_mode():
        for X_test,y_test in test_dataloader:
            test_pred = model_0(X_test)

            test_loss += loss_fn(test_pred, y_test)

            test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

        test_loss /= len(test_dataloader)

        test_acc /= len(test_dataloader)

    print(f"\nTrain loss: {train_loss} | Test loss: {test_loss:.4f} | Test accuracy: {test_acc:.4f}")

end = timer()
total_train_time = print_train_time(start=start,
                                    end=end,
                                    device=str(next(model_0.parameters()).device))


In [ ]:
str(next(model_0.parameters()).device)

In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
    """Returns dictionary containing the results of model"""
    loss, acc = 0, 0
    model.eval()

    with torch.inference_mode():
        for X, y in tqdm(data_loader):
            y_pred = model(X)

            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y,
                               y_pred=y_pred.argmax(dim=1))
        loss/= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__,
            "model_loss": loss.item(),
            "model_acc": acc}

model_0_results = eval_model(model=model_0,
                             data_loader=test_dataloader,
                             loss_fn=loss_fn,
                             accuracy_fn=accuracy_fn)
model_0_results

In [ ]:
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class FashionModelV1(nn.Module):
    def __init__(self,
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape,
                      out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,
                      out_features=output_shape),
            nn.ReLU()
        )

    def forward(self, x:torch.Tensor):
        return self.layer_stack(x)

In [ ]:
next(model_0.parameters()).device

In [ ]:
torch.manual_seed(42)
model_1 = FashionModelV1(input_shape=784,
                         hidden_units=10,
                         output_shape=10)


In [ ]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.1)

In [ ]:
import torch 

def train_step(epochs: int,
               data_loader: torch.utils.data.DataLoader,
               model: nn.Module,
               loss_fn: nn.Module,
               optimizer: nn.Module):
    start = timer()
    
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch} \n----")
        train_loss = 0

        for batch, (X,y) in enumerate(data_loader):
            model.train()

            y_pred = model(X)

            loss=loss_fn(y_pred, y)
            train_loss += loss

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()
            
            if batch % 400 == 0:
                print(f"Trained on {batch * len(X)}/{len(train_dataloader.dataset)} samples")

        train_loss /= len(data_loader)
    
    end = timer()
    print_train_time(start=start, end=end)
        
    

In [ ]:
train_step(epochs=3,
           data_loader=train_dataloader,
           model=model_1,
           loss_fn=loss_fn,
           optimizer=optimizer)

In [ ]:
def test_step(model: nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: nn.Module,
              accuracy_fn):
    start = timer()
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X_test, y_test in data_loader:
            test_pred = model(X_test)

            test_loss += loss_fn(test_pred, y_test)

            test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

        test_loss /= len(test_dataloader)

        test_acc /= len(test_dataloader)

    print(f"\nTrain loss: {train_loss} | Test loss: {test_loss:.4f} | Test accuracy: {test_acc:.4f}")
    end = timer()
    print_train_time(start=start, end=end)

test_step(model=model_1, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn=accuracy_fn)


In [ ]:
model_1_results = eval_model(model_1, test_dataloader, loss_fn, accuracy_fn)
model_1_results

In [ ]:
class FashionMOdelV2(nn.Module):
    def __init__(self,
                 input_shape: int,
                 hidden_units: int, 
                 output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
                      # hyperparameters -> 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,
                      out_features=output_shape)

        )
    
    def forward(self, x):
        x = self.conv_block_1(x)
        # print(x.shape)
        x = self.conv_block_2(x)
        # print(x.shape)
        x = self.classifer(x)
        # print(x.shape)

        return x

In [ ]:
torch.manual_seed(42)
model_2 = FashionMOdelV2(input_shape=1,
                         hidden_units=10,
                         output_shape=10)

In [ ]:
torch.manual_seed(42)

# batch of images
images = torch.randn(size=(32,3,64,64))
test_image = images[0]

print(f"Image batch shape: {images.shape} ")
print(f"Single image shape: {test_image.shape}")
print(f"Test image \n : {test_image}")

In [ ]:
model_2.state_dict()

In [ ]:
# create a single conv2d layer
conv_layer = nn.Conv2d(in_channels=3,
                       out_channels=10,
                       kernel_size=3,
                       stride=1,
                       padding=1)

conv_output = conv_layer(test_image)
conv_output

In [ ]:
test_image.shape, conv_output.shape

In [ ]:
test_image.shape, test_image.unsqueeze(0).shape

In [ ]:
maxpool_layer = nn.MaxPool2d(kernel_size=2)

print(f"Input shape: {test_image.shape}")

max_output = conv_layer(test_image.unsqueeze(0))
print(f"Shape after convolution layer: {max_output.shape}")

max_output = maxpool_layer(max_output)
print(f"Shape after maxpool layer: {max_output.shape}")

In [ ]:
random_tensor = torch.randn(size=(1,2,2))
print(f"Input shape: {random_tensor.shape}")

# conv_output = conv_layer(random_tensor.unsqueeze(dim=0))
# print(f"Conv Layer shape: {conv_output.shape}")

# max_output = maxpool_layer(conv_output)
# print(f"Max pool Layer shape: {max_output.shape}")

In [ ]:
torch.manual_seed(42)
plt.imshow(image.squeeze(), cmap="gray")

In [ ]:
image.shape

In [ ]:
model_2(image.unsqueeze(0))

In [ ]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(),
                            lr=0.1)

In [ ]:
# training model_2
torch.manual_seed(42)
torch.cuda.manual_seed(42)

train_start = timer()

train_step(3, train_dataloader, model_2, loss_fn, optimizer)
test_step(model_2, test_dataloader, loss_fn, accuracy_fn)

train_stop = timer()
print_train_time(start=train_start, end=train_stop)

In [ ]:
model_2_results = eval_model(
    model_2,
    test_dataloader,
    loss_fn,
    accuracy_fn
)

model_2_results

In [ ]:
model_0_results

In [ ]:
# comaptre results
import pandas as pd
compare_results = pd.DataFrame([model_0_results,
                               model_1_results,
                               model_2_results])

compare_results

In [ ]:
compare_results.set_index("model_name")["model_acc"].plot(kind="barh")
plt.xlabel("accuracy (%)")
plt.ylabel("model")

In [ ]:
# make and evaluate

def make_predictions(model: nn.Module,
                     data: list,
                     device: torch.device = device):
    pred_probs = []
    model.eval()
    model.to(device)

    with torch.inference_mode():
        for sample in data:
            sample = torch.unsqueeze(sample, dim=0).to(device)

            pred_logit = model(sample)

            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)

            pred_probs.append(pred_prob.cpu())
        
    return torch.stack(pred_probs)


In [ ]:
import random 
random.seed(42)

test_samples = []
test_labels = []

for sample, label in random.sample(list(test_dataset), k=9):
    test_samples.append(sample)
    test_labels.append(label)

test_samples[0].shape

In [ ]:
plt.imshow(test_samples[0].squeeze(), cmap="gray")
plt.title(class_names[test_labels[0]])

In [ ]:
pred_probs = make_predictions(model_2, test_samples)
pred_probs[:2]

In [ ]:
pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
test_labels

In [ ]:
# plot predictions
plt.figure(figsize=(9,9))
nrows = 3
ncols = 3

for i, sample in  enumerate(test_samples):
    plt.subplot(nrows, ncols, i+1)

    plt.imshow(sample.squeeze(), cmap="gray")

    pred_label = class_names[pred_classes[i]]

    truth_label = class_names[test_labels[i]]

    title = f"Pred: {pred_label} | Truth: {truth_label}"
    
    if pred_label == truth_label:
        plt.title(title, fontsize=10, c="g")
    else:
        plt.title(title, fontsize=10, c="r")
    
    plt.axis(False)

In [ ]:
from tqdm.auto import tqdm

y_preds = []
model_2.eval()
with torch.inference_mode():
    for X, y in tqdm(test_dataloader, desc="Making predictions.."):
        X, y = X.to(device), y.to(device)

        y_logit = model_2(X)

        y_pred = torch.softmax(y_logit.squeeze(), dim=0).argmax(dim=1)

        y_preds.append(y_pred.cpu())


print(y_preds)
y_pred_tensor = torch.cat(y_preds)
y_pred_tensor[:10]


In [ ]:
import torchmetrics, mlxtend 
torchmetrics.__version__, mlxtend.__version__

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmat = ConfusionMatrix(num_classes=len(class_names),
                          task="multiclass")
confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_dataset.targets)

fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(),
    class_names=class_names,
    figsize=(10,7)
)

In [ ]:
# hello from old